In [ ]:
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import trange
from collections import OrderedDict

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoLocator

from function.utilities import get_subset
from function.seqfilter import SeqFilter

# Param

In [ ]:
#####CHANGE HERE#####
algorithm = "VL3-BA" #VLXT, VSL2, VL3-BA, fidpnn
#####CHANGE HERE#####

In [ ]:
protein_subset = ["rbp", "mrbp", "trbp", "snrbp", "snorbp", "ncrbp", "rrbp", "ribosomerbp"]
aa_sort = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]

random_seed = 42 #or None
sample_times = 100

length_filter = False
disorder_filter_length = 40
order_filter_length = 10

figure_1b_path = Path('./output/fig1')

# Load data

In [ ]:
pondr_disorder_ident_path = Path("./output/pondr_disorder_ident")

od_human_df = pd.read_pickle(pondr_disorder_ident_path / "{}_od_human_df.pkl".format(algorithm))

In [ ]:
od_human_df

# Relative ratio stat

In [ ]:
seqfilter = SeqFilter()
def aa_content(df):
    """
    calculate order/disorder amino acid content in all sequences
    
    df: pandas.DataFrame, sequences subset (human, rbp, mrbp)
    
    return: dict, amino acid content info
    """
    order_sequence_all = ""
    disorder_sequence_all = ""
    aa_sort_od = aa_sort + ["*","x","z"] #z(order) x(disorder) filtered

    for index, row in df.iterrows():
        uniprot_id = row["uniprot_id"]
        order_sequence = row["order_sequence"]
        disorder_sequence = row["disorder_sequence"]
        
        od_ident = row['od_ident']
        
        #od_ident length filter
        if length_filter:
            od_ident = seqfilter.length_filter_by_od_ident(od_ident, disorder_filter_length, order_filter_length)
        
        #filter seqeunce by od_ident
        order_sequence = seqfilter.get_seq_from_od_ident(od_ident, order_sequence, 'order')
        disorder_sequence = seqfilter.get_seq_from_od_ident(od_ident, disorder_sequence, 'disorder')
        
        #sum
        order_sequence_all = order_sequence_all + order_sequence
        disorder_sequence_all = disorder_sequence_all + disorder_sequence

    order_sequence_dict = OrderedDict({i: order_sequence_all.count(i) for i in aa_sort_od})
    order_sequence_dict["total"] = sum(order_sequence_dict.values())

    disorder_sequence_dict = OrderedDict({i: disorder_sequence_all.count(i) for i in aa_sort_od})
    disorder_sequence_dict["total"] = sum(disorder_sequence_dict.values())

    return {"order": order_sequence_dict, 
            "disorder": disorder_sequence_dict}

def subset_human_ratio(subset, human):
    """
    calculate amino acid ratio between subset and human
    
    subset: dict, created by aa_content()
    human: dict, created by aa_content()
    
    return: dict, ratio between two sets
    """
    subset_percent = [subset["disorder"][x] / (subset["order"][x] + subset["disorder"][x]) for x in aa_sort]
    human_percent = [human["disorder"][x] / (human["order"][x] + human["disorder"][x]) for x in aa_sort]

    ratio = [np.log(i / j) for i, j in zip(subset_percent, human_percent)]

    return {"aa_sort": aa_sort, 
            "ratio": ratio}

def population(od_subset_df, od_human_df):
    """
    concat func aa_content() and subset_human_ratio()
    
    od_subset_df: pandas.DataFrame, sequences subset (rbp, mrbp)
    od_human_df: pandas.DataFrame, human set
    
    return: dict, ratio between two sets
    """
    ratio_matrix = np.zeros((1, 20))
    human_content = aa_content(od_human_df)
    subset_content = aa_content(od_subset_df)

    subset_human_ratio_dict = subset_human_ratio(subset_content, human_content)
    ratio = subset_human_ratio_dict["ratio"]
    aa_sort = subset_human_ratio_dict["aa_sort"]

    ratio_matrix[0][:] = ratio

    return {"aa_sort": aa_sort, 
            "ratio_matrix": ratio_matrix}


def sample(od_human_df, sample_nums=1542, sample_times=50, random_seed=None):
    """
    smaple subset from human 
    
    od_human_df: pandas.DataFrame, human set
    sample_nums: int, sequence sample numbers per sample
    sample_times: int, sample times
    random_seed: random seed for reproducibility
    
    return: dict, ratio between sampled human set and all human set, 
                  dim(sample_times,20(amino acid))
    """
    ratio_matrix = np.zeros((sample_times, 20))
    human_content = aa_content(od_human_df)
    
    rdm = np.random.RandomState(seed=random_seed)
    rdm_nums = rdm.randint(1000000,size=sample_times)
    
    t = trange(sample_times, leave=True)
    for i in t:
        t.set_description("sampling {} human subset {} times".format(sample_nums, sample_times))
        t.refresh()

        sampled_df = od_human_df.sample(sample_nums, random_state=rdm_nums[i])
        human_sampled_content = aa_content(sampled_df)

        subset_human_ratio_dict = subset_human_ratio(human_sampled_content, human_content)
        ratio = subset_human_ratio_dict["ratio"]
        aa_sort = subset_human_ratio_dict["aa_sort"]

        ratio_matrix[i][:] = ratio

    return {"aa_sort": aa_sort, 
            "ratio_matrix": ratio_matrix}

In [ ]:
output = {}
for i in protein_subset:
    od_subset_df = get_subset(od_human_df, i)
    subset_nums = len(od_subset_df)
    
    population_dict = population(od_subset_df, od_human_df)
    
    sample_nums = len(od_subset_df)
    sample_dict = sample(od_human_df, sample_nums=sample_nums, sample_times=sample_times, random_seed=random_seed)
    
    output['aa_sort'] = population_dict["aa_sort"]
    output[i] = {"subset"       : i,
                 "nums"         : subset_nums,
                 "population"   : population_dict["ratio_matrix"],
                 "human_sample" : sample_dict["ratio_matrix"]}

# Plot

In [ ]:
#PLOT BATCH
def plot(subset, subplot):
    noise = np.random.normal(0, 0.05, (output[subset]["human_sample"].shape))
    for index, i in enumerate(output[subset]["human_sample"]):
        subplot.scatter(X + noise[index], i, color=(0.105, 0.149, 0.192, 0.8), s=2.5, zorder=10)
    subplot.bar(X, output[subset]["population"][0], width=0.75, tick_label=aa_sort, zorder=0, color="white", hatch="//", linewidth=2, edgecolor="#7d7dff",)
    subplot.set_ylim(-1.25,1.25)
    
    subplot.axes.xaxis.set_ticklabels([])
    
    subplot.tick_params(which='major', axis="x",direction="in", length=1)
    subplot.tick_params(which='minor', axis="x",direction="in", length=1)
    subplot.tick_params(which='major', axis="y",direction="in", length=2)
    subplot.tick_params(which='minor', axis="y",direction="in", length=1)
    
    subplot.yaxis.set_major_locator(MultipleLocator(0.5))
    subplot.yaxis.set_minor_locator(MultipleLocator(0.25))
    
    
mpl.rcParams["hatch.linewidth"] = 2
f, ((a0, a1), (a2, a3), (a4, a5), (a6, a7)) = plt.subplots(4, 2, sharex=True, sharey=True, figsize=(12, 15), dpi=200)
X = np.arange(len(aa_sort))

for subset, subplot in zip(protein_subset,[a0,a1,a2,a3,a4,a5,a6,a7]):
    plot(subset, subplot)
    

plt.tight_layout()
plt.subplots_adjust(wspace=0.02, hspace=0.02)

output_path = figure_1b_path / "fig1b_{}.svg".format(algorithm)
f.savefig(output_path)
output_path = figure_1b_path / "fig1b_{}.png".format(algorithm)
f.savefig(output_path)


In [ ]:
# rbp_population_dict = population(od_rbp_df, od_human_df)
# human_sample_for_rbp = sample(od_human_df, sample_nums=sample_nums_for_rbp, sample_times=sample_times, random_seed=random_seed)

# mrbp_population_dict = population(od_mrbp_df, od_human_df)
# human_sample_for_mrbp = sample(od_human_df, sample_nums=sample_nums_for_mrbp, sample_times=sample_times, random_seed=random_seed)

# output = {
#     "aa_sort": rbp_population_dict["aa_sort"],
#     "rbp": {"population": rbp_population_dict["ratio_matrix"],
#             "human_sample": human_sample_for_rbp["ratio_matrix"],
#             },
#     "mrbp": {"population": mrbp_population_dict["ratio_matrix"],
#              "human_sample": human_sample_for_mrbp["ratio_matrix"],
#             },
#     }

In [ ]:
#PLOT 2
def plot(subset, subplot):
    noise = np.random.normal(0, 0.05, (output[subset]["human_sample"].shape))
    for index, i in enumerate(output[subset]["human_sample"]):
        subplot.scatter(X + noise[index], i, color=(0.105, 0.149, 0.192, 0.8), s=2.5, zorder=10)
    subplot.bar(X, output[subset]["population"][0], width=0.75, tick_label=aa_sort, zorder=0, color="white", hatch="//", linewidth=2, edgecolor="#c06be4",)
    subplot.set_ylim(-0.35,0.95)
    
    subplot.axes.xaxis.set_ticklabels([])
    
    subplot.tick_params(which='major', axis="x",direction="in", length=1)
    subplot.tick_params(which='minor', axis="x",direction="in", length=1)
    subplot.tick_params(which='major', axis="y",direction="in", length=2)
    subplot.tick_params(which='minor', axis="y",direction="in", length=1)
    
    subplot.yaxis.set_major_locator(MultipleLocator(0.1))
#     subplot.yaxis.set_minor_locator(MultipleLocator(0.25))
    
    
mpl.rcParams["hatch.linewidth"] = 2
f, ((a0, a1), (a2, a3), (a4, a5), (a6, a7)) = plt.subplots(4, 2, sharex=True, sharey=True, figsize=(12, 15), dpi=200)
X = np.arange(len(aa_sort))

for subset, subplot in zip(protein_subset,[a0,a1]):
    plot(subset, subplot)
    

plt.tight_layout()
plt.subplots_adjust(wspace=0.02, hspace=0.02)

figure_1b_path = Path("/home/wenlin/tmp")
output_path = figure_1b_path / "filter_length_fig1b_{}.svg".format(algorithm)
f.savefig(output_path)
output_path = figure_1b_path / "filter_length_fig1b_{}.png".format(algorithm)
f.savefig(output_path)


In [ ]:
# df = pd.DataFrame()

# df["aa"] = rbp['order'].keys()

# df["rbp_order"] = rbp['order'].values()
# df["rbp_disorder"] = rbp['disorder'].values()

# df["mrbp_order"] = mrbp['order'].values()
# df["mrbp_disorder"] = mrbp['disorder'].values()

# df["human_order"] = human['order'].values()
# df["human_disorder"] = human['disorder'].values()

# df.to_excel(table_s2tos5_path/"{}.xlsx".format(algorithm))
# df

# Table s6 output

In [ ]:
# df = pd.DataFrame()

# df['random_1533_ave'] = np.mean(human_sample_for_rbp['ratio_matrix'],axis=0)
# df['random_1533_std'] = np.std(human_sample_for_rbp['ratio_matrix'],axis=0)
# df['random_1533_max'] = np.max(human_sample_for_rbp['ratio_matrix'],axis=0)
# df['random_1533_min'] = np.min(human_sample_for_rbp['ratio_matrix'],axis=0)

# df['random_688_ave'] = np.mean(human_sample_for_mrbp['ratio_matrix'],axis=0)
# df['random_688_std'] = np.std(human_sample_for_mrbp['ratio_matrix'],axis=0)
# df['random_688_max'] = np.max(human_sample_for_mrbp['ratio_matrix'],axis=0)
# df['random_688_min'] = np.min(human_sample_for_mrbp['ratio_matrix'],axis=0)

# df = round(df,2)

# df.to_excel(table_s6_path/"{}.xlsx".format(algorithm))
# df